So now we'll focus on the JSON/web API - this is basically generating a session, and then "pinging" specific URLs for JSON responses with relevant information. This API is good for when you need to list stuff as text, getting metadata and rendering thumbnails/downsampled versions of your image. It is bad for creating stuff and essentially cannot get raw pixels.

We start the same way as in the intro - config file, helper functions, matplotlib. I'm also importing `json` for formatted outputs. Then, I'm using that same helper function to create a session.

In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT, OMEROWEB
from ezomero.json_api import create_json_session, get_rendered_jpeg
import matplotlib.pyplot as plt
import requests
import json
import warnings
warnings.filterwarnings(action="once") # silence repeated http warnings in JSON API calls

login_rsp, session, base_url = create_json_session(web_host=OMEROWEB, user=OMEROUSER, password=OMEROPASS,
                                                   verify=False) # use verify=False for dev server,
                                                                 # use verify=True for JAX research server

First of all, let's have a look at the list of URLs this API gives us. 

In [ ]:
r = session.get(base_url)
urls = r.json()
print(json.dumps(urls, indent=1))

Ok, let's start by having a look at the omero servers associated to this web interface - in this case it's boring because it's just the one server. But this is an example for the general flow of this section: start from one of the basic URLs we retrieved from the server, add extra bits when necessary, put that into a `get` request and get the json output/print it nicely.

In [ ]:
servers_url = urls['url:servers']
servers_json = session.get(servers_url).json()
print(json.dumps(servers_json, indent=1))

Now let's have a look at all projects from a specific owner. (Here I am getting one specific user id, but alternatively you can list groups by `experimentergroups` URL, get the group ID you want, retrieve experimenters with `experimentergroups/{group_id}/experimenters/` and get the ID you want). Filter projects by owner URL, get JSON, print, as usual.

In [ ]:
projects_url = urls['url:projects']
user1_url = projects_url+"?owner=205"
user1_json = session.get(user1_url).json()
print(json.dumps(user1_json, indent=1))

Ok, so now we get any project id there and have a look at that project in specific.

In [ ]:
myproj_id = 405
myproj_url = projects_url+str(myproj_id)+"/"
myproj_json = session.get(myproj_url).json()
print(json.dumps(myproj_json, indent=1))

Now we can look at data about the individual datasets inside that project.

In [ ]:
data_in_proj_url = myproj_url + "datasets/"
print(data_in_proj_url)
data_in_proj_json = session.get(data_in_proj_url).json()
print(json.dumps(data_in_proj_json, indent=1))

Let's keep digging! I'll get one of those dataset ids and look at it.

In [ ]:
ds_id = 561
data_url = urls['url:datasets'] + str(ds_id) + "/"
print(data_url)
data_json = session.get(data_url).json()
print(json.dumps(data_json, indent=1))

And this is the URL you'd use to get info about each image in that dataset - I won't do it because it's a lot of output!

In [ ]:
images_in_ds_url = data_url + "images/"
print(images_in_ds_url)

I will, though, have a look at one individual image.

In [ ]:
image_id = 39005
image_url = urls['url:images'] + str(image_id) + "/"
print(image_url)
image_json = session.get(image_url).json()
print(json.dumps(image_json, indent=1))

Finally, let's have a look at all the ROIs in this image. If your image doesn't have any ROIs, go to OMERO.web, draw a couple and run this again.

In [ ]:
rois_url = image_url + "rois/"
print(rois_url)
rois_json = session.get(rois_url).json()
print(json.dumps(rois_json, indent=1))

Finally, as a reminder, let's retrieve a jpeg-compressed version of our image!

In [ ]:
scale_factor=1
img = get_rendered_jpeg(session, base_url, image_id, scale_factor)
plt.imshow(img)